<a href="https://colab.research.google.com/github/sdam-au/mounds_ETL/blob/master/scripts/EXPLORING-MOUNDS-IN-GSHEETS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%%capture

### GENERAL REQUIREMENTS - will install the libraries
import numpy as np
import math
import pandas as pd
from google.colab import files
import sys
import requests
from urllib.request import urlopen 
from bs4 import BeautifulSoup
import io
# to avoid errors, we sometime use time.sleep(N) before retrying a request
import time
# the input data have typically a json structure
import json
import getpass
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import warnings
warnings.filterwarnings('ignore')
from functools import reduce


import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from google.colab import auth
from oauth2client.client import GoogleCredentials

!pip install sddk ### our own package under construction, always install to have an up-to-date version
import sddk

In [0]:
### configure session and groupurl
### in the case of "SDAM_root", the folder owner is Vojtech with username 648597@au.dk
conf = sddk.configure_session_and_url("SDAM_root", "648597@au.dk")

sciencedata.dk username (format '123456@au.dk'): 648597@au.dk
sciencedata.dk password: ··········
connection with shared folder established with you as its owner
endpoint variable has been configured to: https://sciencedata.dk/files/SDAM_root/


# Reading and merging the data

In [0]:
### authorize google sheets 
auth.authenticate_user()
gc = gspread.authorize(GoogleCredentials.get_application_default())

# establish connection with particular sheets by their url:
AORDataset = gc.open_by_url("https://docs.google.com/spreadsheets/d/1cx0nntcCLgrwQvCvvIYkjFJ-TnoJ0QRJfTgrf2qEEv4/edit?usp=sharing")
IzvestiaDataset = gc.open_by_url("https://docs.google.com/spreadsheets/d/1wOxbKVHGNHox4h86Z5ZXXADUubMJofC1KqDLMLT13a8/edit?usp=sharing")

# Preprocessing AOR dataset

In [0]:
# what worksheets we have:
AORDataset.worksheets()

[<Worksheet 'GeneralSpatial' id:1795672216>,
 <Worksheet 'MoundAttributes' id:819285652>,
 <Worksheet 'BurialAttributes' id:1240910271>,
 <Worksheet 'Lookups' id:504663605>,
 <Worksheet 'MoundNumbers' id:2134988134>]

In [0]:
# importing three lists into three dataframes
AORDataset_GeneralSpatial = pd.DataFrame(AORDataset.worksheet("GeneralSpatial").get_all_records())
AORDataset_MoundAttributes = pd.DataFrame(AORDataset.worksheet("MoundAttributes").get_all_records())
AORDataset_BurialAttributes = pd.DataFrame(AORDataset.worksheet("BurialAttributes").get_all_records())

In [0]:
AORDataset_BurialAttributes.head(5)

,MoundID,Name,GraveNo,Enclosure Type,LaborAssessment,Burial No,Extra skeletal remains?,Notes on burial,Assemblage?,AnimalRemains,Lithics,HMPottery,Storage,FineVessels,DrinkingCups,Jewellery,Weapons,SpecialFinds,Imports,Grave Rank symbols,StartDate,Enddate,Chronology rating
0,1002,Mound 5,1,Simple pit,no elaboration (eg.a pit or no indistinct encl...,1,N,,N,,,,,,,,,,,0 - No symbols of status present,-3500,-3000,1 - rough estimate by author on basis of the a...
1,1002,Mound 5,1,Simple pit,no elaboration (eg.a pit or no indistinct encl...,2,N,,N,,,,,,,,,,,0 - No symbols of status present,-3500,-3000,1 - rough estimate by author on basis of the a...
2,1002,Mound 5,2,Simple pit,no elaboration (eg.a pit or no indistinct encl...,1,N,,N,,,,,,,,,,,0 - No symbols of status present,-3500,-3000,1 - rough estimate by author on basis of the a...
3,1002,Mound 5,2,Simple pit,no elaboration (eg.a pit or no indistinct encl...,2,N,,N,,,,,,,,,,,0 - No symbols of status present,-3500,-3000,1 - rough estimate by author on basis of the a...
4,1003,No name,1,Simple pit,no elaboration (eg.a pit or no indistinct encl...,1,N,Main Burial (earlier),Y,,Y (2),,,,,2(silver spirals),,red ochre,,"1 - One or two status symbols (ochre, personal...",-3500,-3000,3 - a number of burial aspects consulted and c...


In [0]:
# force column to integers
def get_int(value):
  try:
      variable = int(value)
  except ValueError:
      variable = None
  return variable
for column in ["StartDate", "Enddate"]:
  AORDataset_BurialAttributes[column] = AORDataset_BurialAttributes.apply(lambda row: get_int(row[column]), axis=1)

In [0]:
# grouping BurialAttributes by mounds

def get_data_from_group(group):
  #group = pd.DataFrame(group).reset_index().sort_values("StartDate")
  mound = {"MoundID" : group["MoundID"][0]}
  mound["FirstStartDate"] = group['StartDate'][0]
  mound["FirstEndDate"] = group['Enddate'][0]
  mound["MaxEndDate"] = group["Enddate"].max()
  mound["FirstChronologyRating"] = group["Chronology rating"][0]
  mound["FirstEnclosureType"] = group["Enclosure Type"][0]
  mound["Graves"] = len(group)
  mound["FirstBurialNumber"] = group["Burial No"][0]
  mound["GraveRankSymbols"] = list(set([rank.partition(" -")[0] for rank in group["Grave Rank symbols"].tolist()]))
  mound["AllDates"] = [(date[0], date[1]) for date in zip(group['StartDate'], group['Enddate'])]
  try:
    mound["TotalBurialNumber"] = group["Burial No"].astype(int).sum()
  except:
    mound["TotalBurialNumber"] = None
  mound["BurialAttributesAggr"] = pd.DataFrame(group).reset_index().T.to_dict() #.to_dict()
  return mound
AORDataset_BurialAttributes_grouped = pd.DataFrame(AORDataset_BurialAttributes.sort_values("StartDate").groupby("MoundID").apply(lambda group_df: get_data_from_group(group_df)).tolist())
AORDataset_BurialAttributes_grouped.head(10)

,MoundID,FirstStartDate,FirstEndDate,MaxEndDate,FirstChronologyRating,FirstEnclosureType,Graves,FirstBurialNumber,GraveRankSymbols,AllDates,TotalBurialNumber,BurialAttributesAggr
0,1002,-3500.0,-3000.0,-3000.0,1 - rough estimate by author on basis of the a...,Simple pit,4,1,[0],"[(-3500.0, -3000.0), (-3500.0, -3000.0), (-350...",6.0,"{0: {'index': 0, 'MoundID': 1002, 'Name': 'Mou..."
1,1003,-3500.0,-3000.0,-3000.0,3 - a number of burial aspects consulted and c...,Simple pit,2,1,"[0, 1]","[(-3500.0, -3000.0), (-3500.0, -3000.0)]",2.0,"{0: {'index': 4, 'MoundID': 1003, 'Name': 'No ..."
2,1004,150.0,200.0,250.0,3 - a number of burial aspects consulted and c...,sarcophagos,3,1,[1],"[(150.0, 200.0), (200.0, 225.0), (200.0, 250.0)]",3.0,"{0: {'index': 6, 'MoundID': 1004, 'Name': 'Kra..."
3,1005,100.0,150.0,400.0,"4 - C14 dates acquired, or other scientific ev...",tiled grave,4,1,"[0, 2, 1]","[(100.0, 150.0), (100.0, 150.0), (100.0, 150.0...",4.0,"{0: {'index': 10, 'MoundID': 1005, 'Name': 'Go..."
4,1006,300.0,400.0,500.0,1 - rough estimate by author on basis of the a...,Simple pit,91,1,"[0, 1]","[(300.0, 400.0), (300.0, 400.0), (300.0, 400.0...",91.0,"{0: {'index': 57, 'MoundID': 1006, 'Name': 'Pe..."
5,1008,-900.0,-600.0,-600.0,1 - rough estimate by author on basis of the a...,dolmen,1,1,[0],"[(-900.0, -600.0)]",1.0,"{0: {'index': 104, 'MoundID': 1008, 'Name': 'M..."
6,1009,100.0,300.0,300.0,1 - rough estimate by author on basis of the a...,Simple pit,2,1,[0],"[(100.0, 300.0), (100.0, 300.0)]",2.0,"{0: {'index': 106, 'MoundID': 1009, 'Name': 'R..."
7,1010,-100.0,-50.0,-50.0,3 - a number of burial aspects consulted and c...,clay platform - funeral pyre,10,1,"[0, 1]","[(-100.0, -50.0), (-100.0, -50.0), (-100.0, -5...",16.0,"{0: {'index': 107, 'MoundID': 1010, 'Name': 'M..."
8,1011,-300.0,-280.0,-280.0,3 - a number of burial aspects consulted and c...,Dug in tomb,2,1,"[3, 0]","[(-300.0, -280.0), (-300.0, -280.0)]",2.0,"{0: {'index': 118, 'MoundID': 1011, 'Name': 'M..."
9,1012,-400.0,-200.0,-200.0,1 - rough estimate by author on basis of the a...,Simple pit,2,1,"[2, 1]","[(-400.0, -200.0), (nan, nan)]",2.0,"{0: {'index': 120, 'MoundID': 1012, 'Name': 'C..."


In [0]:
# example
AORDataset_BurialAttributes_grouped[AORDataset_BurialAttributes_grouped["MoundID"]== 1115]["BurialAttributesAggr"].tolist()[0][6]

{'AnimalRemains': '',
 'Assemblage?': 'Y',
 'Burial No': '',
 'Chronology rating': '1 - rough estimate by author on basis of the assemblage',
 'DrinkingCups': '',
 'Enclosure Type': 'Funeral pyre in a pit',
 'Enddate': 250.0,
 'Extra skeletal remains?': '',
 'FineVessels': '',
 'Grave Rank symbols': 'not available',
 'GraveNo': 3,
 'HMPottery': '',
 'Imports': '',
 'Jewellery': '',
 'LaborAssessment': 'some elaboration (marks of a coffin, small stone circle, tiled or stone lined grave of RM period)',
 'Lithics': '',
 'MoundID': 1115,
 'Name': 'Mound 2',
 'Notes on burial': '',
 'SpecialFinds': '',
 'StartDate': 150.0,
 'Storage': '',
 'Weapons': '',
 'index': 820}

In [0]:
# exporting to gsheet
#set_with_dataframe(AORDataset.worksheet("BurialAttributes_grouped"), AORDataset_BurialAttributes_grouped)

In [0]:
# merge using "MoundID"
dfs = [AORDataset_GeneralSpatial, AORDataset_MoundAttributes, AORDataset_BurialAttributes_grouped]
AOR_df = dfs[0].merge(dfs[1], on="MoundID", how="inner").merge(dfs[2], on="MoundID", how="inner")

In [0]:
AOR_df.head(5)

,MoundID,Municipality,Region,Lat,Long,Error radius(m),LU_Around,MoundCover,Geomorphology,Prominence,MoundName,HeightMax,HeightMin,DiameterMax,DiameterMin,GT,RT,Condition,Source of Impact,NumberOfBurialsInside,FirstStartDate,FirstEndDate,MaxEndDate,FirstChronologyRating,FirstEnclosureType,Graves,FirstBurialNumber,GraveRankSymbols,AllDates,TotalBurialNumber,BurialAttributesAggr
0,1107,Sliven,Sliven,42.4916,26.2698,0,Annual agriculture,No data,hillside,"somewhat prominent (on a plateau, slope or spu...",no name,5.6,1.6,36,34,,Y,2 - partially damaged (type of damage is descr...,looting,10,0.0,100.0,400.0,2 - specialist consulted on a single item,,10,1,"[0, 1]","[(0.0, 100.0), (0.0, 100.0), (0.0, 100.0), (0....",10.0,"{0: {'index': 743, 'MoundID': 1107, 'Name': 'n..."
1,1179,Belitsa,Blagoevgrad,41.9357,23.5736,0,Pasture (grassland),Pasture (grassland),on the ridge,very (on an outcrop or peak),Mogilata sas sarkofaga,7,,30,,,N,"1 - pristine, (only superficial or minor damag...",,1,-400.0,-300.0,-300.0,2 - specialist consulted on a single item,Cist grave,1,1,[2],"[(-400.0, -300.0)]",1.0,"{0: {'index': 1190, 'MoundID': 1179, 'Name': '..."
2,1392,Belitsa,Blagoevgrad,41.9198,23.5723,0,No data,No data,on the ridge,"somewhat prominent (on a plateau, slope or spu...",Golyamata mogila,4,,30,25,,N,"1 - pristine, (only superficial or minor damag...",,1,NaN,NaN,NaN,0 - no chronology,Simple pit,1,1,[0],"[(nan, nan)]",1.0,"{0: {'index': 1992, 'MoundID': 1392, 'Name': '..."
3,1393,Blagoevgrad,Blagoevgrad,41.9769,23.0931,100,Scrub,No data,hilltop,very (on an outcrop or peak),Kaymenska chuka,,,,,,Y,2 - partially damaged (type of damage is descr...,looting,4,200.0,300.0,300.0,1 - rough estimate by author on basis of the a...,Tiled grave,4,1,[0],"[(200.0, 300.0), (200.0, 300.0), (200.0, 300.0...",4.0,"{0: {'index': 1993, 'MoundID': 1393, 'Name': '..."
4,1012,Primorsko,Burgas,42.2778,27.7269,200,Forest,Forest,"valley bottom, or flat ground","not prominent (in a valley, plain or depression)",Chenger 1,3.25,2.98,28.6,26.8,,N,"1 - pristine, (only superficial or minor damag...",,2,-400.0,-200.0,-200.0,1 - rough estimate by author on basis of the a...,Simple pit,2,1,"[2, 1]","[(-400.0, -200.0), (nan, nan)]",2.0,"{0: {'index': 120, 'MoundID': 1012, 'Name': 'C..."


In [0]:
# export to gsheet
# (without BurialAttributesAggr column)
set_with_dataframe(AORDataset.add_worksheet("AOR_aggr", 1, 1), AOR_df.drop("BurialAttributesAggr", axis=1))

In [0]:
# export to sciencedata
sddk.write_file("SDAM_data/mounds/AOR_df.json", AOR_df, conf)

Your <class 'pandas.core.frame.DataFrame'> object has been succefully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/mounds/AOR_df.json"


# Preprocessing Izvestia dataset

In [0]:
# what worksheets we have:
IzvestiaDataset.worksheets()

[<Worksheet 'General' id:369527838>,
 <Worksheet 'MoundAttributes' id:548739213>,
 <Worksheet 'BurialAttributes' id:18001881>,
 <Worksheet 'FlatCemeteries' id:34049049>,
 <Worksheet 'TimeOnTask' id:136710821>,
 <Worksheet 'Lookups' id:935910602>]

In [0]:
# importing three lists into three dataframes
IzvestiaDataset_General = pd.DataFrame(IzvestiaDataset.worksheet("General").get_all_records())
IzvestiaDataset_MoundAttributes = pd.DataFrame(IzvestiaDataset.worksheet("MoundAttributes").get_all_records())
IzvestiaDataset_BurialAttributes = pd.DataFrame(IzvestiaDataset.worksheet("BurialAttributes").get_all_records())

In [0]:
# force dates as numbers (see above)
for column in ["StartDate", "Enddate"]:
  IzvestiaDataset_BurialAttributes[column] = IzvestiaDataset_BurialAttributes.apply(lambda row: get_int(row[column]), axis=1)

In [0]:
IzvestiaDataset_BurialAttributes.head(5)

,MoundID,Name,GraveNo,Enclosure Type,Enclosure dimensions,Enclosure materials,LaborAssessment,Burial No,Burial type,Sex,Age,SkeletonOrder,SkeletonPosition,InhumOrientation,Extra skeletal remains?,Notes on burial,Assemblage?,Lithics,CoarsePottery,Storage,FineVessels,DrinkingCups,Jewellery,Weapons,SpecialFinds,Imports,Grave Rank symbols,Notes of burial assemblage,StartDate,Enddate,Date based on,ChronoResource,Chronology rating,Reflections on chronology
0,2,Divdiyadovo m.,1,Urn,,"note, just a dug pit",no elaboration (eg.a pit or no indistinct encl...,1,cremation,male,not available,"incomplete skeleton (due to preservation, or r...",not available,not available,N,,N,N,1(bowl),1 (krater),1 (cup),,1 (clasp/ bucle ?),,die (from red clay),Y,"1 - One or two status symbols (ochre, personal...",,-450.0,-300.0,,"Dremsizova1963, 2-3",1 - rough estimate by author on basis of the a...,there are pictures of the inventory
1,3,Malamir m.,1,Urn,0.55x0.38m,field unworked stones on top,"some elaboration (marks of a coffin, small sto...",1,cremation,uncertain,adolescent,"incomplete skeleton (due to preservation, or r...",not available,not available,N,,N,N,1 (bowl),2 (lekythos;cantharus ),,,1 (fibulae),,,Y,"1 - One or two status symbols (ochre, personal...",,-200.0,-300.0,Fibulae type is middle-laten. ?,"Dremsizova1963, 6-7",1 - rough estimate by author on basis of the a...,there are pictures of the cist grave and of th...
2,3,Malamir m.,2,Tomb of brick or stone,,,,,,,,,,,,,,,,,,,,,,,,,-200.0,-300.0,,"Dremsizova1963, 6-7",1 - rough estimate by author on basis of the a...,no picture
3,4,Kalugeritsa necropolis,1,Urn,"1.12x0.77m, depth=0.60 m",field unworked stones on top,"some elaboration (marks of a coffin, small sto...",1,cremation,uncertain,not available,no comment on skeleton order,not available,not available,N,All the burials are located quate a distance ...,,N,2 (dishes);1 vessel,,,,,2 (knifes),,N,0 - No symbols of status present,,NaN,NaN,,"Damyanov,Popov1972 ,82-83",1 - rough estimate by author on basis of the a...,no picture
4,4,Kalugeritsa necropolis,1,Urn,,,,2,cremation,uncertain,not available,no comment on skeleton order,not available,not available,N,,N,N,2 (dishes);,,,,,,,,,,NaN,NaN,,"Damyanov,Popov1972 ,82-83",1 - rough estimate by author on basis of the a...,there are pictures of the inventory


In [0]:
IzvestiaDataset_BurialAttributes.columns

Index(['MoundID', 'Name', 'GraveNo', 'Enclosure Type', 'Enclosure dimensions',
       'Enclosure materials', 'LaborAssessment', 'Burial No', 'Burial type',
       'Sex', 'Age', 'SkeletonOrder', 'SkeletonPosition', 'InhumOrientation',
       'Extra skeletal remains?', 'Notes on burial', 'Assemblage?', 'Lithics',
       'CoarsePottery', 'Storage', 'FineVessels', 'DrinkingCups', 'Jewellery',
       'Weapons', 'SpecialFinds', 'Imports', 'Grave Rank symbols',
       'Notes of burial assemblage', 'StartDate', 'Enddate', 'Date based on',
       'ChronoResource', 'Chronology rating', 'Reflections on chronology'],
      dtype='object')

In [0]:
IzvestiaDataset_BurialAttributes_grouped = pd.DataFrame(IzvestiaDataset_BurialAttributes.sort_values("StartDate").groupby("MoundID").apply(lambda group_df: get_data_from_group(group_df)).tolist())
IzvestiaDataset_BurialAttributes_grouped.head(10)

,MoundID,FirstStartDate,FirstEndDate,MaxEndDate,FirstChronologyRating,FirstEnclosureType,Graves,FirstBurialNumber,GraveRankSymbols,AllDates,TotalBurialNumber,BurialAttributesAggr
0,2,-450.0,-300.0,-300.0,1 - rough estimate by author on basis of the a...,Urn,1,1,[1],"[(-450.0, -300.0)]",1.0,"{0: {'index': 0, 'MoundID': 2, 'Name': 'Divdiy..."
1,3,-200.0,-300.0,-300.0,1 - rough estimate by author on basis of the a...,Tomb of brick or stone,2,,"[, 1]","[(-200.0, -300.0), (-200.0, -300.0)]",NaN,"{0: {'index': 2, 'MoundID': 3, 'Name': 'Malami..."
2,4,-450.0,-350.0,-350.0,1 - rough estimate by author on basis of the a...,Urn,6,1,"[, 0]","[(-450.0, -350.0), (nan, nan), (nan, nan), (na...",8.0,"{0: {'index': 8, 'MoundID': 4, 'Name': 'Kaluge..."
3,5,-450.0,-400.0,-400.0,1 - rough estimate by author on basis of the a...,not available,1,,[not available],"[(-450.0, -400.0)]",NaN,"{0: {'index': 9, 'MoundID': 5, 'Name': 'Diyado..."
4,6,-600.0,-450.0,-450.0,3 - a number of burial aspects consulted and c...,Tomb of brick or stone,2,1,"[0, 1]","[(-600.0, -450.0), (-600.0, -450.0)]",3.0,"{0: {'index': 10, 'MoundID': 6, 'Name': 'Mound..."
5,7,-600.0,-450.0,-450.0,3 - a number of burial aspects consulted and c...,Tomb of brick or stone,2,1,"[, AA]","[(-600.0, -450.0), (-600.0, -450.0)]",3.0,"{0: {'index': 12, 'MoundID': 7, 'Name': 'Mound..."
6,8,-600.0,-450.0,-450.0,3 - a number of burial aspects consulted and c...,Tomb of brick or stone,2,1,"[0, 2]","[(-600.0, -450.0), (-600.0, -450.0)]",3.0,"{0: {'index': 14, 'MoundID': 8, 'Name': 'Mound..."
7,9,-600.0,-450.0,-450.0,3 - a number of burial aspects consulted and c...,Tomb of brick or stone,4,1,"[0, 1]","[(-600.0, -450.0), (-600.0, -450.0), (-600.0, ...",10.0,"{0: {'index': 16, 'MoundID': 9, 'Name': 'Mound..."
8,10,-600.0,-450.0,-450.0,3 - a number of burial aspects consulted and c...,Tomb of brick or stone,3,1,"[0, 1]","[(-600.0, -450.0), (-600.0, -450.0), (-600.0, ...",6.0,"{0: {'index': 20, 'MoundID': 10, 'Name': 'Moun..."
9,11,-600.0,-450.0,-450.0,3 - a number of burial aspects consulted and c...,Tomb of brick or stone,3,1,"[0, 1]","[(-600.0, -450.0), (-600.0, -450.0), (-600.0, ...",6.0,"{0: {'index': 23, 'MoundID': 11, 'Name': 'Moun..."


In [0]:
# uncomment to export to dataframe 
#set_with_dataframe(IzvestiaDataset.worksheet("BurialAttributes_grouped"), IzvestiaDataset_BurialAttributes_grouped)

In [0]:
# merge using "MoundID"
dfs = [IzvestiaDataset_General, IzvestiaDataset_MoundAttributes, IzvestiaDataset_BurialAttributes_grouped]
Izvestia_df = dfs[0].merge(dfs[1], on="MoundID", how="inner").merge(dfs[2], on="MoundID", how="inner")

In [0]:
Izvestia_df

,MoundID,MoundName_x,Location description in text,Municipality,Region,Lat,Long,LocPrecision,Source,Error radius(m),LU_Around,MoundCover,Prominence,Reason for excavation,Current status,PhotoExists,DrawingExists,Asociated settlement,Distance to assoc.settlement (m),References,,BG Code,Mound Date,Bara Notes,MoundName_y,BiblioReference,Elev_msl,HeightMin,HeightMax,Dim1,Dim2,GT,Notes on Dimensions,RT,Condition,Source of Impact,NumberOfBurialsInside,Graves_x,HumanBurialsRemains,Cenotaph?,AnimalRemainsNoHuman,Notes on Mound Condition and Grave number,FirstStartDate,FirstEndDate,MaxEndDate,FirstChronologyRating,FirstEnclosureType,Graves_y,FirstBurialNumber,GraveRankSymbols,AllDates,TotalBurialNumber,BurialAttributesAggr
0,2,Divdyadovo m.,Diuzmeshe area,Divdyadovo,Shumen,N/A,N/A,not available,,the author did not specify the distance from ...,agricultural field,crops,"somewhat prominent (on a plateau, slope or spu...",Research - Large attractive mound,"Partially excavated, still stands",Y,Y,N,N/A,Dremsizova 1963,,,,I have no idea where is the given village and ...,Divdyadovo m.,"Dremsizova 1963, 2-3",N/A,N/A,1,10,12,,,N,2 - partially damaged (type of damage is descr...,ploughing,1,1,1,,,The number of graves is 1 cremation,-450.0,-300.0,-300.0,1 - rough estimate by author on basis of the a...,Urn,1,1,[1],"[(-450.0, -300.0)]",1.0,"{0: {'index': 0, 'MoundID': 2, 'Name': 'Divdiy..."
1,3,Malamir m.,"2 km west, next to the Malamir village",Malamir v.,Shumen,N/A,N/A,not available,,the author did not specify the distance from ...,agricultural field,crops,not available,,"Partially excavated, still stands",Y,Y,N,2000,,,,,I have no idea where is the given village and ...,Malamir m.,"Dremsizova 1963, 6-7",N/A,0.8,1,10,12,,,N,2 - partially damaged (type of damage is descr...,ploughing,2 (1 urn and 1 tomb),2,1,Y,,The number of graves is 1 cremation and one d...,-200.0,-300.0,-300.0,1 - rough estimate by author on basis of the a...,Tomb of brick or stone,2,,"[, 1]","[(-200.0, -300.0), (-200.0, -300.0)]",NaN,"{0: {'index': 2, 'MoundID': 3, 'Name': 'Malami..."
2,4,Kalugeritsa necropolis,Kanara bashi area,Kalugeritsa v.,Shumen,N/A,N/A,not available,,the author did not specify the distance from ...,agricultural field,crops,"not prominent (in a valley, plain or depression)",,"Partially excavated, and removed",Y,N,N,N/A,"Damyanov,Popov 1972",,,,I have no idea where is the given village and ...,Kalugeritsa necopolis,"Damyanov,Popov1972 ,81-94",N/A,N/A,N/A,N/A,N/A,,,N,5 - nearly extinct (archaeologically useful cu...,communications (road on or next to mound),4 graves (6 burials),4,6,,,"The number of graves is 4 , but there are 6 bu...",-450.0,-350.0,-350.0,1 - rough estimate by author on basis of the a...,Urn,6,1,"[, 0]","[(-450.0, -350.0), (nan, nan), (nan, nan), (na...",8.0,"{0: {'index': 8, 'MoundID': 4, 'Name': 'Kaluge..."
3,5,Dyadozlatyuvata m.,Goliyam Yurdan area,Smyadovo,Shumen,43.0516,26.9796,3 - legacy reports (very coarse),Topo 50,7500,agricultural field,crops,not available,,not available,Y,N,N,N/A,Dremsizova 1963,,,,I don't have the publication,Dyadozlatyuvata m.,"Dremsizova 1963, 8-10",N/A,N/A,N/A,N/A,N/A,,,N,2 - partially damaged (type of damage is descr...,ploughing,NA,NA,NA,,,,-450.0,-400.0,-400.0,1 - rough estimate by author on basis of the a...,not available,1,,[not available],"[(-450.0, -400.0)]",NaN,"{0: {'index': 9, 'MoundID': 5, 'Name': 'Diyado..."
4,6,Mound 1,"Kurchan tepe area,Dobrina necropolis",Dobrina v.,Provadiya,43.2076,27.4598,3 - legacy reports (very coarse),"Topo 50 / Google Earth / Mirchev 1965, 34 - 35",150,stony field,thin black earth,"somewhat prominent (on a plateau, slope or spu...",Research - Large attractive mound,Excavated and removed,Y,N,Y(other 27 mounds of the necropolis),N/A,Mirchev 1965,,,,,Mound 1,"Mirchev1965, 35-36",N/A,0.15,0.2,7.5,N/A,,,N,not available,not available,1 grave (2 burials),1,2,,,"Small mound, cover with stones, and a couple o...",-600.0,-450.0,-450.0,3 - a number of burial aspects consulted and c.

In [0]:
# export to gsheet
set_with_dataframe(IzvestiaDataset.add_worksheet("Izvestia_aggr", 1,1), Izvestia_df.drop("BurialAttributesAggr", axis=1))

In [0]:
sddk.write_file("SDAM_data/mounds/Izvestia_df.json", Izvestia_df, conf)

Your <class 'pandas.core.frame.DataFrame'> object has been succefully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/mounds/Izvestia_df.json"
